In [1]:
import os
import sys
import datetime
import numpy as np
import pandas as pd
from myIOUtils import *
from myDatetime import *

sys.path.append('/home/aistudio/data')
# 原始数据需提前解压出来
data_folder_path = '/home/aistudio/data/data124390/data_unzip_2018'
# 所有输出结果保存在work工作目录下
result_folder_path = '/home/aistudio/work'

# 自定义文件存储位置
failure_serial_file = "failure_serial_result.txt"
success_serial_file = "success_serial_result.txt"

# 读取所有的文件
dataFiles = os.listdir(data_folder_path) 
# data_len = 365, data_count = 36700506, failure count == 1378

### 设置测试文件数量 ###
EXAMPLE_NUM = 10
dataFiles = dataFiles[:EXAMPLE_NUM]

data_size = len(dataFiles)
print("成功加载的数据量为：" + str(data_size))
if data_size < 365 :
    print("！！！当前加载的为测试数据！！！")


成功加载的数据量为：10
！！！当前加载的为测试数据！！！


In [ ]:
#######################################
### 该模块用于获取出现故障的硬盘序列号 ###
#######################################

failure_serial_list = []
failure_count = 0 # 出现过故障的硬盘总数，由serial_number来确定唯一性
for index, fileName in enumerate(dataFiles):
    filePath = data_folder_path + "/" + fileName
    all_df = pd.read_csv(filePath)
    # 选择列属性'failure'值为1的数据切片
    all_df = all_df.loc[all_df['failure']==1]
    # 切片所有数据的'serial_number'列
    all_df = all_df.loc[:, "serial_number"]
    all_df_list = all_df.values.tolist()
    for serial_item in all_df_list:
        if serial_item in failure_serial_list:
            print("重复出现故障的serial_number : " + serial_item) # 输出重复出现的serial_number
        else:
            failure_serial_list.append(serial_item)
            failure_count = failure_count + 1
    # 跟踪数据处理进度
    if (index+1)%36 == 0:
        print((str(((index+1)/36)*10)) + "% data finished...")
    elif(index == 364):
        print("!!! all finished !!!")
print("failure count == "+ str(failure_count))
# print(failure_serial_list)

write_list_to_file(failure_serial_list, result_folder_path, failure_serial_file)


默认硬盘序列号serial number为唯一标识符

ZA13C001 ZA14LWDQ 5641SFS9S 该标识出现过两次

In [ ]:
#######################################
### 该模块用于提取所有出故障硬盘数据集 ###
#######################################

# 加载failure_serial_list
failure_serial_list = read_file_to_list(result_folder_path, failure_serial_file)
print("总故障硬盘数量为："+str(len(failure_serial_list)))

# 提取故障硬盘的所有数据信息
for _ , fileName in enumerate(dataFiles):
    filePath = data_folder_path + "/" + fileName
    all_df = pd.read_csv(filePath)
    # 检查序列号是否出现在故障列表中，获取故障硬盘数据集
    failure_df = all_df.loc[all_df['serial_number'].isin(failure_serial_list)]
    # 通过文件名获取时间序列信息
    time_interval = time_interval_from_fileName(fileName)
    # 在原数据列表中插入时间序列
    failure_df.insert(loc=1, column='time_interval', value=time_interval)
    # 写入csv文件中
    failure_csv_file = result_folder_path + "/failure/date/" + fileName
    with open(failure_csv_file, 'w+') as f:
        failure_df.to_csv(f)
    print(fileName + ' 输出故障硬盘数量 --> ' + str(len(failure_df)))


In [ ]:
#########################################
### 该模块用于故障硬盘数据集的序列号分类 ###
#########################################

# 只加载故障故障硬盘的数据集
failure_dict = {}
for _ , fileName in enumerate(dataFiles):
    failure_csv_file = result_folder_path + "/failure/date/" + fileName
    with open(failure_csv_file, 'r') as f:
        failure_df = pd.read_csv(f)
        # 故障序列号列表
        failure_serial_list = failure_df.iloc[:,3].values.tolist()
        # print(failure_serial_list)
        for serial_item in failure_serial_list:
            current_df = failure_df.loc[failure_df['serial_number'] == serial_item]
            if serial_item not in failure_dict.keys():
                # value信息改为直接保存df数据
                failure_dict[serial_item] = [current_df]
            else:
                failure_dict[serial_item].append(current_df)
    print(fileName)
# 检查failure_dict键数量是否等于
print(len(failure_dict.keys()))

for key in failure_dict.keys():
    df_data_list = failure_dict[key]
    df_data = pd.DataFrame() # 初始化DataFrame数据结构
    for df_item in df_data_list:
        df_data = df_data.append(df_item)
    # 对df按照interval_time的列递增排序写入csv
    df_data = df_data.sort_values(by=['time_interval'], ascending=True) 
    # 写入csv文件中
    clean_csv_file = result_folder_path + "/failure/clean/" + key + ".csv"
    with open(clean_csv_file, 'w+') as f:
        df_data.to_csv(f)
    print(key + " -->>> ")

In [2]:
#########################################
### 提取无故障硬盘的数据集serial_num列表 ###
#########################################
'''
    解决思路：
    1.读取2018-12-31的硬盘序列号
    2.提取不在故障列表中且从第一天就存在的硬盘序列（最为robust的硬盘）
'''
# 获取特定某一天的serial_list, selected_date = 2018-01-01
def get_serial_list_by_selected_date(selected_date):
    selected_date_file_path = data_folder_path + "/" + selected_date + '.csv'
    selected_date_df = pd.read_csv(selected_date_file_path)
    selected_serial_list = selected_date_df.loc[:,"serial_number"].values.tolist() # serial_number列切片
    return selected_serial_list

# 加载故障硬盘信息用于对比
failure_serial_list = read_file_to_list(result_folder_path, failure_serial_file)
print("总故障硬盘数量为："+str(len(failure_serial_list)))

first_day_serial_list = get_serial_list_by_selected_date("2018-01-01")
last_day_serial_list = get_serial_list_by_selected_date("2018-12-31")

# 从第一天到最后一天都没有故障的硬盘数量 78844
total_success_count = 0
success_serial_list = []
for serial_item in last_day_serial_list:
    if serial_item not in failure_serial_list and serial_item in first_day_serial_list:
        success_serial_list.append(serial_item)
        total_success_count = total_success_count + 1

# 保存无故障的硬盘serial_num信息
write_list_to_file(success_serial_list, result_folder_path, success_serial_file)

总故障硬盘数量为：1378


KeyboardInterrupt: 

In [2]:
###################################
###提取无故障硬盘数据集的详细信息 ###
###################################

from random import sample

# 加载无故障的硬盘序列
success_serial_list = read_file_to_list(result_folder_path, success_serial_file)

# 随机抽取列表的2000个元素(考虑故障数据集的size为1300左右)
SELECTED_SIZE = 2
selected_serial_list = sample(success_serial_list, SELECTED_SIZE)
print("selectd_success_data_size == " + str(len(selected_serial_list)))

'''
解决思路：
1.遍历所有原始文件，找出每个文件中无故障的硬盘数据，同时插入time_interval列构成按天的success_df
2.构建success_dict，其中key值为硬盘serial_number，但value值使用当天的df数据
3.遍历success_dict的key值（保存的serial_num）,对该硬盘的所有df值按照时间系列time_interval递增排序
'''
success_dict = {} # key: serial_num || value: df 详细信息
for _ , fileName in enumerate(dataFiles):
    filePath = data_folder_path + "/" + fileName
    with open(filePath, 'r') as f:
        success_df = pd.read_csv(f)
        # 截取serial_number值存在于selected_serial_list的df数据
        success_df = success_df.loc[success_df['serial_number'].isin(selected_serial_list)]
        # 通过文件名获取时间序列信息
        time_interval = time_interval_from_fileName(fileName)
        # 在原success_df中插入时间列time_interval
        success_df.insert(loc=1, column='time_interval', value=time_interval)
        success_serial_list = success_df.iloc[:,2].values.tolist()
        for serial_item in success_serial_list:
            current_df = success_df.loc[success_df['serial_number'] == serial_item]
            # 遍历无故障的硬盘serial，将value值设置为存储df结构的list
            if serial_item not in success_dict.keys():
                success_dict[serial_item] = [current_df]
            else:
                success_dict[serial_item].append(current_df)
    print(fileName)

for key in success_dict.keys():
    df_data_list = success_dict[key]
    df_data = pd.DataFrame() # 初始化DataFrame数据结构
    for _, df_item in enumerate(df_data_list):
        df_data = df_data.append(df_item)
    # 对df按照interval_time的列递增排序写入csv
    df_data = df_data.sort_values(by=['time_interval'], ascending=True) 
    success_csv_file = result_folder_path + "/success/clean/" + key + ".csv"
    with open(success_csv_file, 'w+') as f:
        df_data.to_csv(f)
    print(key + " -->>> ")

selectd_success_data_size == 2
2018-07-07.csv
2018-10-04.csv
2018-01-23.csv
2018-06-12.csv
2018-12-15.csv
2018-03-30.csv
2018-05-29.csv
2018-10-02.csv
2018-08-10.csv
2018-10-11.csv
PL1331LAGS3XKH -->>> 
Z304JPP9 -->>> 
